## langgraph-sdk: Time traveling to a node with interrupt

In [1]:
from langgraph_sdk import get_client
from langgraph_sdk.schema import Command
client = get_client(url="http://localhost:2024")

# Using the graph deployed with the name "agent"
assistant_id = "Music Catalog Subagent"

# create a thread
thread = await client.threads.create()
thread_id = thread["thread_id"]
input = {"messages": [{"role": "human", "content": "what albums do you have by the rolling stones?"}]}

# Run the graph until the interrupt is hit.
result = await client.runs.wait(
    thread_id,
    assistant_id,
    input=input   # (1)!
)


In [2]:

print(result['__interrupt__'])

[{'value': 'continue?', 'id': 'b26b130c9efc34257003e485e703dede'}]


In [3]:
# Resume the graph

print(await client.runs.wait(
    thread_id,
    assistant_id,
    command=Command(resume=True)
))

{'messages': [{'content': 'what albums do you have by the rolling stones?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'ec092b11-9189-4f0d-8607-37a408391d4d'}, {'content': '', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 92, 'prompt_tokens': 475, 'total_tokens': 567, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-DCeDoR9KEtC9soDwwLsslLQepIINA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'lc_run--019c8de1-edcd-7682-b3ab-bc4f7a9ebae1-0', 'tool_calls': [{'name': 'get_albums_by_artist', 'args': {'artist': 'The Rolling Stones'}, 'id': 'call_kRc1jjYEePfN4sJWxakGY6Rp

In [4]:
# The states are returned in reverse chronological order.
states = await client.threads.get_history(thread['thread_id'])
selected_state = states[4]
print(selected_state['values'])
print(selected_state['interrupts'])
print(selected_state["checkpoint_id"])

{'messages': [{'content': 'what albums do you have by the rolling stones?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'ec092b11-9189-4f0d-8607-37a408391d4d'}]}
[{'id': 'b26b130c9efc34257003e485e703dede', 'value': 'continue?'}]
1f111384-d0b3-6752-8000-8f718516cd3e


## If you resume using checkpoint_id, interrupt is not triggered as expected

In [5]:
result = await client.runs.wait(
    thread_id,
    assistant_id,
    checkpoint_id=selected_state["checkpoint_id"]
)


## Update state to create a new fork, then invoke the run. Interrupt is triggered as expected

In [6]:
new_config = await client.threads.update_state(
    thread_id,
    values=selected_state['values'],
    checkpoint_id=selected_state["checkpoint_id"]
)
print(new_config)

{'checkpoint': {'thread_id': '923b2b19-df27-4548-863e-5363536c5b25', 'checkpoint_ns': '', 'checkpoint_id': '1f111388-91af-6b4c-8001-9847c52a0289'}, 'configurable': {'thread_id': '923b2b19-df27-4548-863e-5363536c5b25', 'checkpoint_ns': '', 'checkpoint_id': '1f111388-91af-6b4c-8001-9847c52a0289'}, 'checkpoint_id': '1f111388-91af-6b4c-8001-9847c52a0289'}


In [7]:

result = await client.runs.wait(
    thread_id,
    assistant_id,
    input=None,
    checkpoint_id=new_config["checkpoint_id"]  # Use the new one
)

In [12]:
print(result)
print(result['__interrupt__'])

{'messages': [{'content': 'what albums do you have by the rolling stones?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'ec092b11-9189-4f0d-8607-37a408391d4d'}], '__interrupt__': [{'value': 'continue?', 'id': '18703e2b26d4c40f071d3083f78ca3fb'}]}
[{'value': 'continue?', 'id': '18703e2b26d4c40f071d3083f78ca3fb'}]
